In [ ]:
#%%writefile productos_vdcns.py
######## PROTOCOLO AUTOMATICO PARA LA GENERACION DE INDICES APLICADOS #######
#######        A AGUAS CONTINENTALES CON LANDSAT 8 Y SENTINEL 2        ######
######                                                                  #####
####                        Autor: Diego Garcia Diaz                     ####
###                      email: diegogarcia@ebd.csic.es                   ###
##                  GitHub: https://github.com/Digdgeo/VDCNS               ##
#                        Sevilla 01/08/2016-28/02/2018                      #

# coding: utf-8

import os, shutil, re, time, subprocess, pandas, rasterio, sys, urllib, fiona, sqlite3, math, ogr, shapely, pymongo
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal, gdalconst
from datetime import datetime, date
from shapely.geometry import mapping, Polygon, MultiLineString, shape
from rasterstats import zonal_stats
from scipy import ndimage
from fiona.crs import from_epsg


class Product(object):
    
    
    '''Esta clase genera los productos necesarios para el proyecto (Clorofila, Ficocianina, Turbidez y cota 
    del agua en el embalse. Estos valores se incluiran en la base de datos de las escenas usadas en el proyecto)'''
    
    def __init__(self, ruta_nor):
        
        self.shape = shape
        self.ruta_escena = ruta_nor
        self.escena = os.path.split(self.ruta_escena)[1]
        self.nor = os.path.split(self.ruta_escena)[0]
        self.raiz = os.path.split(self.nor)[0] 
        #print(self.raiz)
        self.nor = os.path.join(self.raiz, os.path.join('nor', self.escena))
        #print(self.nor)
        self.pro = os.path.join(self.raiz, 'pro')
        #print(self.pro)
        self.pro_escena = os.path.join(self.pro, self.escena)
        if not os.path.exists(self.pro_escena):
            os.makedirs(self.pro_escena)
        self.ori = os.path.join(self.raiz, os.path.join('ori', self.escena))
        self.data = os.path.join(self.raiz, 'data')
        self.temp = os.path.join(self.data, 'temp')
        self.productos = os.path.join(self.raiz, 'productos')
        self.dtm_mtn = os.path.join(self.data, 'dtm_mtn.tif')
        self.dtm_lidar = r'C:\Users\Diego\Desktop\VDCNS\LIDAR\MDT05_LIDAR.tif' #os.path.join(self.data, 'lidar_318.tif')
        self.dtm_2006 = os.path.join(self.data, 'mdt_2006_rec.tif')
        self.vals = {}
        self.d = {}
        self.cota = 0
        self.pro_esc = os.path.join(self.productos, self.escena)
        if not os.path.exists(self.pro_esc):
            os.makedirs(self.pro_esc)
            
        if 'l8oli' in self.escena:
            self.sat = 'L8'
        elif 'l7etm' in self.escena:
            self.sat = 'L7'
        elif 'l5tm' in self.escena:
            self.sat = 'L5'
        elif 'l4tm' in self.escena:
            self.sat = 'L4'
        else:
            print('no reconozco el satelite')

        if self.sat == 'L8':

            for i in os.listdir(self.nor):
                if re.search('img$', i):
                    
                    banda = i[-6:-4]
                                        
                    if banda == 'b2':
                        self.b2 = os.path.join(self.nor, i)
                    elif banda == 'b3':
                        self.b3 = os.path.join(self.nor, i)
                    elif banda == 'b4':
                        self.b4 = os.path.join(self.nor, i)
                    elif banda == 'b5':
                        self.b5 = os.path.join(self.nor, i)
                    elif banda == 'b6':
                        self.b6 = os.path.join(self.nor, i)
                    elif banda == 'b7':
                        self.b7 = os.path.join(self.nor, i)
                   
        else:

            for i in os.listdir(self.nor):
                if re.search('img$', i):
                    
                    banda = i[-6:-4]
                                        
                    if banda == 'b1':
                        self.b1 = os.path.join(self.nor, i)
                    elif banda == 'b2':
                        self.b2 = os.path.join(self.nor, i)
                    elif banda == 'b3':
                        self.b3 = os.path.join(self.nor, i)
                    elif banda == 'b4':
                        self.b4 = os.path.join(self.nor, i)
                    elif banda == 'b5':
                        self.b5 = os.path.join(self.nor, i)
                    elif banda == 'b7':
                        self.b7 = os.path.join(self.nor, i)
                        

            
                        
                        
    def get_stats(self):
        
        '''Este metodo se encarga de extraer los valores estadisticos de un shape con respecto a un raster'''
        print('Getting statistics')
        
        for i in os.listdir(self.pro_escena):
            if i.endswith('_WaterMask.shp'):
        
                shape = os.path.join(self.pro_escena, i)
        
        
        stats_mtn = zonal_stats(shape, self.dtm_mtn, \
                    stats=['majority', 'median', 'mean', 'max', 'min', 'range'])
        
        stats_lidar = zonal_stats(shape, self.dtm_lidar, \
                    stats=['majority', 'median', 'mean', 'max', 'min', 'range'])
        
        stats_2006 = zonal_stats(shape, self.dtm_2006, \
                    stats=['majority', 'median', 'mean', 'max', 'min', 'range'])
        
        print(stats_mtn[0], '\n', stats_lidar[0], '\n', stats_2006[0], '\n')
        
        self.cota = stats_mtn[0]['median']
        
        ##Insertamos los datos en MongoDB
        connection = pymongo.MongoClient("mongodb://localhost")
        db=connection.teledeteccion
        vdcns = db.vdcns
        
        try:
        
            vdcns.update_one({'_id':self.escena}, {'$set':{'Cota': {'MTN': stats_mtn[0], 'LIDAR': stats_lidar[0],\
                                                                    'JEX2006': stats_2006[0]}}}) 
            
        except Exception as e:
            
            print("Unexpected error:", type(e), e) 
                        
class get_water_level(Product):
    
    '''En esta clase se implementa todo el codigo necesario para llevar a cabo la generacion 
    de las laminas de agua con su cota asignada'''
                        
    def get_water_rec(self, shape):

        '''Primero hacemos el recorte a la cota 318, habria que introducir la ruta del shape
        con el cual queramos hacer el recorte'''
                
        if self.sat == 'L8':
            banda = self.b6

        else:
            banda = self.b5
            print(banda)
                    
            #except Exception as e:
                #print('Error', e)
                #print('No ha NIR noramlizado en la escena', self.escena)
                
        #print('banda', banda, self.sat)     
        #shape = r'O:\VDCNS\protocolo\data\cota_318p.shp'
        salida = os.path.join(self.pro_escena, self.escena + '_water_rec_b5.img')
        cmd = ["gdalwarp", "-dstnodata" , "0" , "-cutline", "-crop_to_cutline", "-tr", "30", "30", "-of", "ENVI", "-tap"]
        
        cmd.append(banda)
        cmd.append(salida)
        cmd.insert(4, shape) #seria el 4/2 con/sin el dst nodata
        #print(cmd)
        proc = subprocess.Popen(cmd,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        stdout,stderr=proc.communicate()
        exit_code=proc.wait()

        if exit_code: 
            raise RuntimeError(stderr)
        else:
            print(stdout)
            print('recorte de la banda', banda[-6:-4], 'generado')
                

    def reclass_water(self):

        '''Hacemos el reclassify a las bandas recortadas para '''
        reclass = os.path.join(self.pro_escena, self.escena + '_water_reclass.img')
        print(reclass)
        
        for i in os.listdir(self.pro_escena):
            
            if re.search('_water_rec_b..img$', i):

                raster = os.path.join(self.pro_escena, i)

        with rasterio.open(raster) as src:
            B5 = src.read()

            B5[(B5 <= 1450) & (B5 > 0)] = 1
            B5[B5 > 1450] = 0
            
            profile = src.meta
            profile.update(dtype=rasterio.int16)

        with rasterio.open(reclass, 'w', **profile) as dst:
            dst.write(B5.astype(rasterio.int16))
            
            
    def reclass_water_3classes(self):

        '''Hacemos el reclassify a las bandas recortadas para '''
        reclass = os.path.join(self.pro_escena, self.escena + '_water_reclass_3.img')
        print(reclass)
        
        for i in os.listdir(self.pro_escena):
            
            if re.search('_water_rec_b..img$', i):

                raster = os.path.join(self.pro_escena, i)

        with rasterio.open(raster) as src:
            B5 = src.read()

            B5[(B5 <= 1450) & (B5 > 0)] = 1
            B5[B5 > 1450] = 2
            
            profile = src.meta
            profile.update(dtype=rasterio.int16)

        with rasterio.open(reclass, 'w', **profile) as dst:
            dst.write(B5.astype(rasterio.int16))

    
    def get_edges(self):
        
        '''Este metodo sirve para vecorizar la lamina de agua obtenida'''
        
        out = os.path.join(self.pro_escena, self.escena + '_edges.img')
        print(out)
        for i in os.listdir(self.pro_escena):
            if re.search('reclass_3.img$', i):
                b5 = os.path.join(self.pro_escena, i)
        
        with rasterio.open(b5) as src:
            B5 = src.read()

        B4 = B5.reshape(453, 484)
        B3 = np.where(B4 != 1, 0, B4)
                
        mask = ((B3 != 1))
        mask2 = ((B4 != 1))
        
        struct = ndimage.generate_binary_structure(2, 2)
        erode = ndimage.binary_erosion(B3, struct)
        edges = mask ^ erode
        #e = edges.reshape(1, 453, 484)
        
        #Probamos a hacer la ternaria
        struct2 = ndimage.generate_binary_structure(2, 2)
        erode2 = ndimage.binary_erosion(B4, struct2)
        edges2 = mask2 ^ erode2
        #e2 = edges2.reshape(1, 453, 484)

        edges3 = np.where((edges2 == 1) & (edges == 0), 1, 0)
        e3 = edges3.reshape(1, 453, 484)

        profile = src.meta
        profile.update(dtype=rasterio.int32)

        with rasterio.open(out, 'w', **profile) as dst:
            dst.write(e3.astype(rasterio.int32))

    
    def polygonize(self):
        
        '''Este metodo sirve para vecorizar la lamina de agua obtenida'''
            
        outShp = os.path.join(self.pro_escena, self.escena + '_poly.shp')

        for i in os.listdir(self.pro_escena):
            if re.search('water_reclass.img$', i):
                print(i)
                water = os.path.join(self.pro_escena, i)

        sourceRaster = gdal.Open(water)
        band = sourceRaster.GetRasterBand(1)
        bandArray = band.ReadAsArray()

        driver = ogr.GetDriverByName("ESRI Shapefile")
        if os.path.exists(outShp):
            driver.DeleteDataSource(outShp)
        outDatasource = driver.CreateDataSource(outShp)
        outLayer = outDatasource.CreateLayer("polygonized", srs= None)
        gdal.Polygonize( band, None, outLayer, -1, [], callback=None )
        outDatasource.Destroy()
        sourceRaster = None
            
    def get_vector_mask_pg(self):
        
        '''Este metodo sirve para seleccionar la lamina de agua vectorial adecuada (la que tenga la segunda mayor area)'''
        
        out = os.path.join(self.pro_escena, self.escena[:8] + '_watervec.shp')
        
        for i in os.listdir(self.pro_escena):
            if i.endswith('poly.shp'):
                shp = os.path.join(self.pro_escena, i)
                
        myshp = fiona.open(shp)

        areas = []
        selection = []

        for i in myshp.values():

            geom1 = i['geometry']
            a1 = Polygon(geom1['coordinates'][0])
            #print('\nArea', a1.area, '\n')
            areas.append(a1.area)

        #print(sorted(areas))

        for i in myshp.values():

            geom1 = i['geometry']
            a1 = Polygon(geom1['coordinates'][0])

            if a1.area == sorted(areas)[-2]:
                selection.append(i)


        with fiona.open(shp, 'r') as source:

            # **source.meta is a shortcut to get the crs, driver, and schema
            # keyword arguments from the source Collection.
            with fiona.open(out, 'w', **source.meta) as sink:


                for f in source:

                    geom1 = f['geometry']
                    a1 = Polygon(geom1['coordinates'][0])
                    if a1.area == sorted(areas)[-2]:
                        sink.write(f)
        
    def get_vector_mask_pl(self, ratio):
        
        '''En este metodo elegimos el poligono de la mascara de agua y 
        lo pasamos a vectorial (suavizando tambien la geometria)'''
        
        outshp = os.path.join(self.pro_escena, self.escena[:10] + '_WaterMask.shp')
        
        for i in os.listdir(self.pro_escena):
            
            if i.endswith('_watervec.shp'):
                
                shape = os.path.join(self.pro_escena, i)
                print(shape)
                myshp = fiona.open(shape)
            
        for i in myshp:
            geom1 = i['geometry']
            #print(geom1)
            #print(geom1)
            line = MultiLineString(geom1['coordinates'])
            line10 = line.simplify(ratio)
        

        # Define a polygon feature geometry with one attribute
        schema = {
            'geometry': 'MultiLineString',
            'properties': {'id': 'int'}}

        # Write a new Shapefile
        with fiona.open(outshp, 'w', 'ESRI Shapefile', schema) as c:
            ## If there are multiple geometries, put the "for" loop here
            c.write({
                'geometry': mapping(line10),
                'properties': {'id': 123},
            })
                
    def get_contour(self):
        
        '''En este metodo creamos una copia de la linea de agua con la cota ya asignada'''
        
        outshp = os.path.join(self.pro_escena, self.escena[:8] + '_contour.shp')
        
        for i in os.listdir(self.pro_escena):
            if i.endswith('_WaterMask.shp'):
                myshp = fiona.open(os.path.join(self.pro_escena, i))
        
        for i in myshp:
            
            geom1 = i['geometry']
            #print(geom1)
            #print(geom1)
            line = MultiLineString(geom1['coordinates'])
            

        # Define a polygon feature geometry with one attribute
        schema = {
            'geometry': 'MultiLineString',
            'properties': {'id': 'str', 'cota': 'int'}}
        
        # Write a new Shapefile
        with fiona.open(outshp, 'w', crs=from_epsg(25830),driver='ESRI Shapefile', schema=schema) as c:
            ## If there are multiple geometries, put the "for" loop here
            c.write({
                'geometry': mapping(line),
                'properties': {'id': self.escena, 'cota': self.cota},
            })

                    
    def run_water_mask(self):
        
        self.get_water_rec(r'O:\VDCNS\protocolo\data\cota_318p.shp')
        self.reclass_water()
        self.polygonize()
        self.get_vector_mask_pg()
        self.get_vector_mask_pl(0)
        self.get_stats()
        self.reclass_water_3classes()
        self.get_edges()
        self.get_contour()
        
        

In [ ]:
shape = r'O:\VDCNS\protocolo\data\cota_318p.shp'
raster = r'O:\VDCNS\protocolo\pro\20020616l7etm202_32'

a = get_water_level(raster)
a.run_water_mask()

In [ ]:
'''a.get_water_rec(shape)
a.reclass_water()
a.polygonize()
a.get_vector_mask_pg()
a.get_vector_mask_pl(20)'''
a.get_stats()

In [ ]:
ruta = r'O:\VDCNS\protocolo\nor'
shape = r'O:\VDCNS\protocolo\data\cota_318p.shp'

for i in os.listdir(ruta):
    
    try:
        
        a = get_water_level(os.path.join(ruta, i))
        a.run_water_mask()
        
    except Exception as e:
        print(e)
        continue

In [ ]:
rutanor = r'O:\VDCNS\protocolo\nor'

suma = 0
b6 = 0
b5 = 0

for i in os.listdir(rutanor):
    ndir = os.path.join(rutanor, i)
    total = 0
    for f in os.listdir(ndir):
        total += 1
        if total == 19:
            suma += 1
        elif 'l8oli' in ndir:
            if f.endswith('b6.img'):
                b6 += 1
        else:
            if f.endswith('b5.img'):
                b5 += 1